In [1]:
import numpy as np
from scipy.linalg import svd
import math 
import sys
import random

In [2]:
def calculateEquations(objp, imgp):
    A = np.zeros((2*len(objp),9), dtype=float)
    for i in range(len(objp)):
        A[2*i] = [objp[i][0], 
                  objp[i][1],  
                  1.0, 
                  0, 0, 0, 
                  -imgp[i][0]*objp[i][0],
                  -imgp[i][0]*objp[i][1],
                  -imgp[i][0]]
        A[(2*i)+1] = [0, 0, 0, 
                     objp[i][0], 
                     objp[i][1],  
                     1.0,
                     -imgp[i][1]*objp[i][0],
                     -imgp[i][1]*objp[i][1],
                     -imgp[i][1]]

    # Perform SVD
    U, D, VT = svd(A)
    # Calculate H
    H = VT[-1]

    # Calculate Vij
    V01 = [H[0]*H[1], H[0]*H[4]+H[3]*H[1] ,H[3]*H[4], H[6]*H[1]+H[0]*H[7], H[6]*H[4]+H[3]*H[7], H[6]*H[7]]
    V00 = [H[0]*H[0], H[0]*H[3]+H[3]*H[0], H[3]*H[3], H[6]*H[0]+H[0]*H[6], H[6]*H[3]+H[3]*H[6], H[6]*H[6]]
    V11 = [H[1]*H[1], H[1]*H[4]+H[4]*H[1], H[4]*H[4], H[7]*H[1]+H[1]*H[7], H[7]*H[4]+H[4]*H[7], H[7]*H[7]]
    numpuSubRes = np.subtract(V00,V11)
    return H, V01, numpuSubRes

In [3]:
def calculateMatrixS(V12, V11_22, V34, V33_44, V56, V55_66):
    V = np.array([[V12],[V11_22],[V34],[V33_44],[V56],[V55_66]])
    V = np.squeeze(V)
    U, D, VT = svd(V)
    S = VT[-1]

    S11, S12, S22, S13, S23, S33 = S[0], S[1], S[2], S[3], S[4], S[5]
    return S11, S12, S22, S13, S23, S33

In [4]:
def calculateMatrixH(Hx):
    h1 = np.matrix([
        [Hx[0]],
        [Hx[3]],
        [Hx[6]]
    ])
    h2 = np.matrix([
        [Hx[1]],
        [Hx[4]],
        [Hx[7]]
    ])
    h3 = np.matrix([
        [Hx[2]],
        [Hx[5]],
        [Hx[8]]
    ])
    return h1, h2, h3

In [5]:
def calcIntrensicParameters(S11, S12, S22, S13, S23, S33):
    C1 = ((S12*S13) - (S11*S23))
    C2 = ((S11*S22) - (S12*S12))
    V0 = C1/C2
    lam = S33 - (((S13*S13)+(V0*C1))/S11)
    alphaU = (lam/S11)**0.5
    alphaV = ((lam*S11)/C2)**0.5
    s = ((-S12*alphaU*alphaU*alphaV)/lam)
    U0 = (((s*V0)/alphaU)-((S13*alphaU*alphaU)/lam))
    Kstar = np.matrix([
        [alphaU, s, U0],
        [0, alphaV, V0],
        [0,    0,    1]
    ])
    KstarInv = np.linalg.inv(Kstar)
    return U0, V0, alphaU, alphaV, s, Kstar, KstarInv

In [6]:
def calcExtrensicParameters(h1i1, h2i1, h3i1, KstarInv):    
    modAlpha = 1 / np.linalg.norm(np.matmul(KstarInv, h1i1))
    signAlpha = np.sign((KstarInv*h3i1)[2])[0,0]
    alpha = signAlpha * modAlpha
    r1 = (alpha * np.matmul(KstarInv, h1i1))
    r2 = (alpha * np.matmul(KstarInv, h2i1))
    r3 = np.matrix(np.cross(r1.T,r2.T))
    flatr1 = [item for sublist in np.array(r1) for item in sublist]
    flatr2 = [item for sublist in np.array(r2) for item in sublist]
    flatr3 = [item for sublist in np.array(r3) for item in sublist]
    Tarray = (alpha * np.matmul(KstarInv, h3i1))
    Tstar = [item for sublist in np.array(Tarray) for item in sublist]
    Rstar = np.array([
        [flatr1[0],flatr2[0],flatr3[0]],
        [flatr1[1],flatr2[1],flatr3[1]],
        [flatr1[2],flatr2[2],flatr3[2]]
    ])
    return Rstar, Tstar

In [7]:
def meanSquareError(H, objp, imgp):
    h1 = H[:3]
    h2 = H[3:6]
    h3 = H[6:9]
    sqerr = 0
    for i, j in zip(objp, imgp):
        xi = j[0]
        yi = j[1]
        pi = np.array(i)
        pih = np.concatenate([pi, [1]])
        xiestimate = (h1.T.dot(pih)) / (h3.T.dot(pih))
        yiestimate = (h2.T.dot(pih)) / (h3.T.dot(pih))
        sqerr = sqerr + ((xi - xiestimate) ** 2 + (yi - yiestimate) ** 2)
    mse = sqerr / len(imgp)
    return mse

In [8]:
def dispIntrensicParams(u0, v0, alphaU, alphaV, s):
    print('(u0,v0)          = ' + str(u0) + " , " + str(v0) )
    print('(alphaU, alphaV) = ' + str(alphaU) + " , " + str(alphaV))
    print(' s               = ' + str(s))
    print("==========================================================================================")

In [9]:
def dispExtrensicParams(tstar, rstar, num):
    print('Image' + str(num) + ":")
    print('\nRstar = ', str(rstar[0][0]), str(rstar[0][1]), str(rstar[0][2]))
    print('        ', str(rstar[1][0]), str(rstar[1][1]), str(rstar[1][2]))
    print('        ', str(rstar[2][0]), str(rstar[2][1]), str(rstar[2][2]))
    print('\nTstar = ', str(tstar[0]))
    print('        ', str(tstar[1]))
    print('        ', str(tstar[2]))
    print("==========================================================================================")

## Camera Callibration

In [10]:
objectPoints0 = np.loadtxt('../data/textFiles/calibrationdata0.txt',usecols = (0, 1))
imagePoints0 = np.loadtxt('../data/textFiles/calibrationdata0.txt',usecols = (3, 4))
objectPoints1 = np.loadtxt('../data/textFiles/calibrationdata1.txt',usecols = (0, 1))
imagePoints1 = np.loadtxt('../data/textFiles/calibrationdata1.txt',usecols = (3, 4))
objectPoints2 = np.loadtxt('../data/textFiles/calibrationdata2.txt',usecols = (0, 1))
imagePoints2 = np.loadtxt('../data/textFiles/calibrationdata2.txt',usecols = (3, 4))
H1, V12, V11_22 = calculateEquations(objectPoints0, imagePoints0)
H2, V34, V33_44 = calculateEquations(objectPoints1, imagePoints1)
H3, V56, V55_66 = calculateEquations(objectPoints2, imagePoints2)
S11, S12, S22, S13, S23, S33 = calculateMatrixS(V12, V11_22, V34, V33_44, V56, V55_66) 
h1i1, h2i1, h3i1 = calculateMatrixH(H1)
h1i2, h2i2, h3i2 = calculateMatrixH(H2)
h1i3, h2i3, h3i3 = calculateMatrixH(H3)
u0, v0, alphaU, alphaV, s, Kstar, KstarInv = calcIntrensicParameters(S11, S12, S22, S13, S23, S33)
Rstari1, Tstari1 = calcExtrensicParameters(h1i1, h2i1, h3i1, KstarInv)
Rstari2, Tstari2 = calcExtrensicParameters(h1i2, h2i2, h3i2, KstarInv)
Rstari3, Tstari3 = calcExtrensicParameters(h1i3, h2i3, h3i3, KstarInv)
dispIntrensicParams(u0, v0, alphaU, alphaV, s)
dispExtrensicParams(Tstari1, Rstari1, 1)
dispExtrensicParams(Tstari2, Rstari2, 2)
dispExtrensicParams(Tstari3, Rstari3, 3)

(u0,v0)          = 544.9279257549949 , 381.0022749464423
(alphaU, alphaV) = 952.3820448477614 , 943.4187001914473
 s               = 1.8173327102012815
Image1:

Rstar =  -0.8425829027520126 0.5090208855434368 0.17539622362187368
         -0.17527707708332707 -0.5672258657589488 0.8046436664342123
         0.5092465004682093 0.6473268802684418 0.5671545094796189

Tstar =  2.515442007330661
         5.797327548552454
         23.325295309423293
Image2:

Rstar =  -0.9920542601009686 -0.005711627764796298 0.12559657175983852
         0.07205115954902717 -0.8512981142531746 0.519832884341464
         0.10313571361638058 0.524590214545251 0.8449454502641489

Tstar =  4.845560157904968
         4.625042565883247
         34.71938177855337
Image3:

Rstar =  -0.8714819519277838 0.42296064316347437 0.24752225329404032
         -0.48433454577015583 -0.8223568923031072 -0.2987449839612769
         0.07706656368203293 -0.38020421027615886 0.9215236406708125

Tstar =  2.066967086663892
         7.55

In [11]:
mse1 = meanSquareError(H1, objectPoints0, imagePoints0)
mse2 = meanSquareError(H2, objectPoints1, imagePoints1)
mse3 = meanSquareError(H3, objectPoints1, imagePoints2)
print('mse1',mse1)
print('mse2',mse2)
print('mse3',mse3)

mse1 0.16725956897977698
mse2 0.04516003716317788
mse3 0.025901091309016685


## Validation

In [12]:
vobjectPoints1 = np.loadtxt('../data/textFiles/profdata3d1.txt',usecols = (0, 1))
vimagePoints1 = np.loadtxt('../data/textFiles/profdata2d1.txt',usecols = (0, 1))
vimagePoints2 = np.loadtxt('../data/textFiles/profdata2d2.txt',usecols = (0, 1))
vimagePoints3 = np.loadtxt('../data/textFiles/profdata2d3.txt',usecols = (0, 1))
H1, V12, V11_22 = calculateEquations(vobjectPoints1, vimagePoints1)
H2, V34, V33_44 = calculateEquations(vobjectPoints1, vimagePoints2)
H3, V56, V55_66 = calculateEquations(vobjectPoints1, vimagePoints3)
S11, S12, S22, S13, S23, S33 = calculateMatrixS(V12, V11_22, V34, V33_44, V56, V55_66) 
h1i1, h2i1, h3i1 = calculateMatrixH(H1)
h1i2, h2i2, h3i2 = calculateMatrixH(H2)
h1i3, h2i3, h3i3 = calculateMatrixH(H3)
u0, v0, alphaU, alphaV, s, Kstar, KstarInv = calcIntrensicParameters(S11, S12, S22, S13, S23, S33)
Rstar1, Tstari1 = calcExtrensicParameters(h1i1, h2i1, h3i1, KstarInv)
Rstar2, Tstari2 = calcExtrensicParameters(h1i2, h2i2, h3i2, KstarInv)
Rstar3, Tstari3 = calcExtrensicParameters(h1i3, h2i3, h3i3, KstarInv)
dispIntrensicParams(u0, v0, alphaU, alphaV, s)
dispExtrensicParams(Tstari1, Rstar1, 1)
dispExtrensicParams(Tstari2, Rstar2, 2)
dispExtrensicParams(Tstari3, Rstar3, 3)

(u0,v0)          = 319.9991020450845 , 239.9982648275653
(alphaU, alphaV) = 1304.3493974889295 , 1304.348884301188
 s               = -0.0007131915461616466
Image1:

Rstar =  -0.4472136265295159 0.8944271632495162 -1.1855703657692729e-07
         0.6666660603069079 0.333333034536825 -0.6666674725845276
         -0.5962854486579267 -0.29814290608529403 -0.7453553083723404

Tstar =  -89.44224549785918
         -199.99892465937654
         849.7068890093235
Image2:

Rstar =  -0.6000001109855112 0.7999999119224105 -8.072175664008974e-08
         0.4997554536435224 0.3748165734326175 -0.7808693418858565
         -0.6246954084759478 -0.46852169687940654 -0.6246943045563418

Tstar =  -39.99952075385564
         -174.91355252579936
         858.9565971305186
Image3:

Rstar =  -0.24253555506183952 0.970142518768906 1.1657523960040272e-07
         0.7995555021188924 0.1998888740611215 -0.5663527460041881
         -0.5494429029137161 -0.1373605789191681 -0.8241629477422598

Tstar =  -145.52088996

In [13]:
mse1 = meanSquareError(H1, vobjectPoints1, vimagePoints1)
mse2 = meanSquareError(H2, vobjectPoints1, vimagePoints2)
mse3 = meanSquareError(H3, vobjectPoints1, vimagePoints3)
print('mse1',mse1)
print('mse2',mse2)
print('mse3',mse3)

mse1 1.6037478658105047e-09
mse2 1.5877812740401116e-09
mse3 1.3648170932051296e-09


## Q3. RANSAC

In [14]:
with open('../data/textFiles/RANSAC.config', 'r') as conf:
    prob = float(conf.readline().split()[0])
    kmax = int(conf.readline().split()[0])
    nmin = int(conf.readline().split()[0])
    nmax = int(conf.readline().split()[0])
    
print(prob, kmax, nmin, nmax)

0.99 1000 6 10


In [15]:
def calcDistance(H, objp, imgp):
    h1 = H[:3]
    h2 = H[3:6]
    h3 = H[6:9]
    sqerr = 0
    distance = []
    for obj, img in zip(objp, imgp):
        xi = img[0]
        yi = img[1]
        pi = np.array(obj)
        pi = np.append(pi, 1)
        xiestimate = (h1.T.dot(pi)) / (h3.T.dot(pi))
        yiestimate = (h2.T.dot(pi)) / (h3.T.dot(pi))
        distancePointPair = (((xi - xiestimate) ** 2 + (yi - yiestimate) ** 2) ** 0.5)
        distance.append(distancePointPair)
    medianDistance = np.median(distance)
    return distance, medianDistance

In [16]:
def ransac(op, ip, prob, nmin, nmax, kmax):
    w = 0.5
    k = kmax
    np.random.seed(20442409)
    count = 0
    inlinerNum = 0
    bestM = None
    H1, V12, V11_22 = calculateEquations(vobjectPoints1, vimagePoints1)
    d, median = calcDistance(H1, nobjectPoints1, nimagePoints1)
    
    threshold = median + 0.5 * median
    n = random.randint(nmin, nmax)
    while(count < k and count < kmax):
        index = np.random.choice(len(op), n)
        ranOp, ranIp = np.array(op)[index], np.array(ip)[index]
        Hrandom, V12, V11_22 = calculateEquations(ranOp, ranIp)
        d, m = calcDistance(Hrandom, op, ip)
        inliner = []
        for i, d in enumerate(d):
            if d < threshold:
                inliner.append(i)
        if len(inliner) >= inlinerNum:
            inlinerNum = len(inliner)
            inlinerOp, inlinerIp = np.array(op)[inliner], np.array(ip)[inliner]
            Hbest, V12, V11_22 = calculateEquations(ranOp, ranIp)
            bestM = Hbest
        if not (w == 0 ):
            w = float(len(inliner))/float(len(ip))
            k = float(math.log(1 - prob)) / np.absolute(math.log(1 - (w ** n)))
        count += 1;
    return inlinerNum, bestM

In [17]:
nobjectPoints1 = np.loadtxt('../data/textFiles/profnoise13d.txt',usecols = (0, 1))
nimagePoints1 = np.loadtxt('../data/textFiles/profnoise12d.txt',usecols = (0, 1))

In [18]:
a, b = ransac(nobjectPoints1, nimagePoints1, prob, nmin, nmax, kmax)
V01 = [b[0]*b[1], b[0]*b[4]+b[3]*b[1], b[3]*b[4], b[6]*b[1]+b[0]*b[7], b[6]*b[4]+b[3]*b[7], b[6]*b[7]]
V00 = [b[0]*b[0], b[0]*b[3]+b[3]*b[0], b[3]*b[3], b[6]*b[0]+b[0]*b[6], b[6]*b[3]+b[3]*b[6], b[6]*b[6]]
V11 = [b[1]*b[1], b[1]*b[4]+b[4]*b[1], b[4]*b[4], b[7]*b[1]+b[1]*b[7], b[7]*b[4]+b[4]*b[7], b[7]*b[7]]
numpuSubRes = np.subtract(V00,V11)
H2, V34, V33_44 = calculateEquations(vobjectPoints1, vimagePoints2)
H3, V56, V55_66 = calculateEquations(vobjectPoints1, vimagePoints3)
S11, S12, S22, S13, S23, S33 = calculateMatrixS(V01, numpuSubRes, V34, V33_44, V56, V55_66) 
h1i1, h2i1, h3i1 = calculateMatrixH(b)
h1i2, h2i2, h3i2 = calculateMatrixH(H2)
h1i3, h2i3, h3i3 = calculateMatrixH(H3)
u0, v0, alphaU, alphaV, s, Kstar, KstarInv = calcIntrensicParameters(S11, S12, S22, S13, S23, S33)
Rstar1, Tstari1 = calcExtrensicParameters(h1i1, h2i1, h3i1, KstarInv)
Rstar2, Tstari2 = calcExtrensicParameters(h1i2, h2i2, h3i2, KstarInv)
Rstar3, Tstari3 = calcExtrensicParameters(h1i3, h2i3, h3i3, KstarInv)
dispIntrensicParams(u0, v0, alphaU, alphaV, s)
dispExtrensicParams(Tstari1, Rstar1, 1)
dispExtrensicParams(Tstari2, Rstar2, 2)
dispExtrensicParams(Tstari3, Rstar3, 3)

(u0,v0)          = 309.8956635600748 , -1090.3589373852471
(alphaU, alphaV) = 1827.7401488164978 , 737.3897106652671
 s               = 3.3548940017270352
Image1:

Rstar =  -0.47146094346668693 0.9530353947329991 -0.03722381426756516
         0.23881859387460758 0.0830841628435085 -1.0217918203733243
         -0.8489347784166088 -0.4512079561846284 -0.26677351068420685

Tstar =  -94.39507925388592
         1753.001943749625
         1267.735446962982
Image2:

Rstar =  -0.5410063580612957 0.7134807800489178 -2.2477669980869308e-07
         -0.3049322671430996 -0.22869955573187928 -0.8772469175887789
         -0.7837910646289753 -0.5878434747820352 0.34129122556008407

Tstar =  -32.713990535378585
         1556.1528464268797
         1077.7132288798186
Image3:

Rstar =  -0.24718726651785944 0.9660987672873236 4.0263645591231967e-07
         0.5911321896354009 0.14778341388789018 -0.7891809266681966
         -0.7677637590094266 -0.1919407346085111 -0.6076222578262424

Tstar =  -141.527242

In [19]:
nobjectPoints2 = np.loadtxt('../data/textFiles/profnoise23d.txt',usecols = (0, 1))
nimagePoints2 = np.loadtxt('../data/textFiles/profnoise22d.txt',usecols = (0, 1))
a2, b2 = ransac(nobjectPoints2, nimagePoints2, prob, nmin, nmax, kmax)

In [20]:
V01 = [b2[0]*b2[1], b2[0]*b2[4]+b2[3]*b2[1], b2[3]*b2[4], b2[6]*b2[1]+b2[0]*b2[7], b2[6]*b2[4]+b2[3]*b2[7], b2[6]*b2[7]]
V00 = [b2[0]*b2[0], b2[0]*b2[3]+b2[3]*b2[0], b2[3]*b2[3], b2[6]*b2[0]+b2[0]*b2[6], b2[6]*b2[3]+b2[3]*b2[6], b2[6]*b2[6]]
V11 = [b2[1]*b2[1], b2[1]*b2[4]+b2[4]*b2[1], b2[4]*b2[4], b2[7]*b2[1]+b2[1]*b2[7], b2[7]*b2[4]+b2[4]*b2[7], b2[7]*b2[7]]
numpuSubRes = np.subtract(V00,V11)
H2, V34, V33_44 = calculateEquations(vobjectPoints1, vimagePoints2)
H3, V56, V55_66 = calculateEquations(vobjectPoints1, vimagePoints3)
S11, S12, S22, S13, S23, S33 = calculateMatrixS(V01, numpuSubRes, V34, V33_44, V56, V55_66) 
h1i1, h2i1, h3i1 = calculateMatrixH(b2)
h1i2, h2i2, h3i2 = calculateMatrixH(H2)
h1i3, h2i3, h3i3 = calculateMatrixH(H3)
u0, v0, alphaU, alphaV, s, Kstar, KstarInv = calcIntrensicParameters(S11, S12, S22, S13, S23, S33)
Rstar1, Tstari1 = calcExtrensicParameters(h1i1, h2i1, h3i1, KstarInv)
Rstar2, Tstari2 = calcExtrensicParameters(h1i2, h2i2, h3i2, KstarInv)
Rstar3, Tstari3 = calcExtrensicParameters(h1i3, h2i3, h3i3, KstarInv)
dispIntrensicParams(u0, v0, alphaU, alphaV, s)
dispExtrensicParams(Tstari1, Rstar1, 1)
dispExtrensicParams(Tstari2, Rstar2, 2)
dispExtrensicParams(Tstari3, Rstar3, 3)

(u0,v0)          = 319.99820985741405 , 239.9408216130002
(alphaU, alphaV) = 1304.3785332790249 , 1304.3523334655688
 s               = -0.0012611474717532099
Image1:

Rstar =  -0.44721370629107027 0.8944270699632682 -4.501489263009084e-07
         0.6666526857667029 0.33332645308569875 -0.6666843563972729
         -0.5963003416613525 -0.2981509446408396 -0.7453403669027673

Tstar =  -89.44175386584081
         -199.96539293988798
         849.7264608457813
Image2:

Rstar =  -0.6000003182217494 0.7999997353647477 -8.0725146733851e-08
         0.49973777490242793 0.3748033143702773 -0.7808867580854609
         -0.6247093519944854 -0.46853215452144625 -0.6246721955664429

Tstar =  -39.9990059870284
         -174.87916515353422
         858.9757694776698
Image3:

Rstar =  -0.24253559617968035 0.9701425134221344 1.1658014051341148e-07
         0.7995470537825251 0.19988676198346633 -0.5663654022794266
         -0.5494551786756139 -0.1373636478563625 -0.8241542433419272

Tstar =  -145.52038

In [21]:
msein1 = meanSquareError(b, vobjectPoints1, vimagePoints1)
msein2 = meanSquareError(b2, vobjectPoints1, vimagePoints1)
print('mse1',msein1)
print('mse2',msein2)


mse1 3.4711478180636504
mse2 3.1755674523544406e-09


In [22]:
#Inliers
a, a2

(66, 111)